In [1]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import numpy as np
from scipy.spatial.distance import cosine

## Loading csv data

In [4]:
data_file = "./data/dummy_docs.csv.txt"
processed_database = pd.read_csv(data_file)

In [5]:
processed_database

,Title,Content
0,Columbia University,Columbia University also known Columbia offici...
1,Fictional University,Fictional University officially private resear...
2,Third Document,Related private term third dissimilar prepared


## Some quick glances at the data

In [15]:
titles = processed_database["Title"]
contents = processed_database["Content"]

In [11]:
titles

0     Columbia University
1    Fictional University
2          Third Document
Name: Title, dtype: object

In [26]:
contents

0    Columbia University also known Columbia offici...
1    Fictional University officially private resear...
2       Related private term third dissimilar prepared
Name: Content, dtype: object

In [32]:
# ngram_range - lower and upper bounds on ngram lengths used
# performs some preprocessing - e.g. by default makes everything lowercase
count_vectorizer = CountVectorizer(ngram_range=(1,1), stop_words='english')
# more efficient than fit and then transform
counted_data = count_vectorizer.fit_transform(contents)

In [30]:
document_term_matrix=pd.DataFrame(counted_data.toarray(), columns=count_vectorizer.get_feature_names_out(), index=titles)

In [37]:
document_term_matrix

,1754,belong,church,city,college,colleges,colonial,columbia,declaration,dissimilar,...,seven,states,term,trinity,united,universities,university,wave,world,york
Title,,,,,,,,,,,,,,,,,,,,,
Columbia University,1,1,1,2,1,1,1,5,1,0,...,1,1,0,1,1,1,3,0,1,3
Fictional University,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,1,0,0
Third Document,0,0,0,0,0,0,0,0,0,1,...,0,0,1,0,0,0,0,0,0,0


In [31]:
document_term_matrix.columns

Index(['1754', 'belong', 'church', 'city', 'college', 'colleges', 'colonial',
       'columbia', 'declaration', 'dissimilar', 'education', 'established',
       'fictional', 'fifth', 'founded', 'george', 'grounds', 'higher',
       'independence', 'institution', 'ivy', 'king', 'known', 'league',
       'learning', 'manhattan', 'new', 'officially', 'old', 'oldest',
       'prepared', 'prior', 'private', 'ranked', 'related', 'research',
       'seven', 'states', 'term', 'trinity', 'united', 'universities',
       'university', 'wave', 'world', 'york'],
      dtype='object')

## Computing tf-idf

In [38]:
tfidf_vectorizer=TfidfVectorizer(use_idf=True, smooth_idf=False) 
documents_TFIDF = pd.DataFrame(tfidf_vectorizer.fit_transform(contents).toarray(), index=titles, columns = tfidf_vectorizer.get_feature_names_out())

In [39]:
documents_TFIDF

,1754,also,among,belong,church,city,college,colleges,colonial,columbia,...,third,top,trinity,united,universities,university,wave,which,world,york
Title,,,,,,,,,,,,,,,,,,,,,
Columbia University,0.102547,0.102547,0.102547,0.102547,0.102547,0.205094,0.102547,0.102547,0.102547,0.512735,...,0.000000,0.102547,0.102547,0.102547,0.102547,0.206031,0.000000,0.102547,0.102547,0.307641
Fictional University,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.275293,0.411062,0.000000,0.000000,0.000000
Third Document,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.437393,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


## Example search

In [35]:
def search(query, idf, idfdf):
    query = idf.transform([query]).toarray()[0] 
    return 1-idfdf.apply(lambda x: cosine(x, query), axis=1).sort_values()[:10]

In [42]:
search("wave", tfidf_vectorizer, documents_TFIDF)

Title
Fictional University    0.411062
Columbia University     0.000000
Third Document          0.000000
dtype: float64